## Ant Colony Optimization algorithm  (ACOA) as an optimizer for training a NN

In [1]:
# importing required libraries
import numpy as np
import pandas as pd
import math
import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt

# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
# importing library for Linear Discriminant Analysis Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing library for Quadratic Discriminant Analysis Model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model

from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model

# representation of model layers
from keras.utils.vis_utils import plot_model
from numpy.random import choice as np_choice
import copy

In [2]:
path="datasets/bin_data.csv"
bin_data=pd.read_csv(path)
bin_data

,Unnamed: 0,count,srv_serror_rate,serror_rate,dst_host_serror_rate,dst_host_srv_serror_rate,logged_in,dst_host_same_srv_rate,dst_host_srv_count,same_srv_rate,...,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,intrusion,abnormal,normal,label
0,0,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-0.782367,-0.818890,0.771283,...,0,0,0,0,1,0,1,0,1,normal
1,1,-0.620982,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-1.161030,-1.035688,-1.321428,...,0,0,0,0,1,0,1,0,1,normal
2,2,0.339648,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.809857,-1.389669,...,1,0,0,0,0,0,0,1,0,abnormal
3,3,-0.690846,-0.184522,-0.189235,-0.572083,-0.602433,1.235694,1.066401,1.258754,0.771283,...,0,0,0,0,1,0,1,0,1,normal
4,4,-0.472521,-0.631929,-0.637209,-0.639532,-0.624871,1.235694,1.066401,1.258754,0.771283,...,0,0,0,0,1,0,1,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,125968,0.872361,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.818890,-1.184947,...,1,0,0,0,0,0,0,1,0,abnormal
125969,125969,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,0.977304,1.159389,0.771283,...,0,0,0,0,1,0,1,0,1,normal
125970,125970,-0.725778,-0.631929,-0.637209,0.979238,-0.624871,1.235694,-0.893738,-0.773724,0.771283,...,0,0,0,0,1,0,1,0,1,normal
125971,125971,0.523041,1.605104,1.602664,1.608759,1.618955,-0.809262,-1.094207,-0.972455,-1.366922,...,1,0,0,0,0,0,0,1,0,abnormal


In [3]:
X = bin_data.iloc[:,0:93].values # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = bin_data[['intrusion']].values # target attribute

In [4]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)

In [5]:
def Fobj():
    mlp = Sequential() # creating model
    # adding input layer and first layer with 50 neurons
    # X_train.shape is (94479, 93)
    mlp.add(Dense(units=33, input_dim=X_train.shape[1], activation='relu'))#50x93=4650
    # output layer with sigmoid activation
    mlp.add(Dense(units=1,activation='sigmoid'))
    return mlp
# len(Fobj().get_weights()[1])
# Fobj().get_weights()[0].flatten().shape[0]

In [6]:
mlp=Fobj()
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dim=0;
for i in range(len(mlp.get_weights())):# 4 for 4 array 
    dim=dim+mlp.get_weights()[i].flatten().shape[0]# 4650+50+50+1=4751
# dim = 4751
# print(math.sqrt(dim))
N=20# Population size
X=np.zeros((N,dim)) #(20, 4751)
# print(X[:,:3])
Fit=np.ones(N)*float('inf') #(20, 1)
# print(Fit.reshape(N,1))
bF=float('inf')
bPos=[];
# print(len(mlp.get_weights()))#4
# len(mlp.layers)#2

In [7]:
# get wights and bias Y=XW+bias 
def setWeights(X):
    Len=0
    # X is each row data in datasets X and len is 4751
    # len(mlp.layers is 2 each contain 4 array
    for i in range(0,len(mlp.layers)):
#         get Wights first from layers
        a=mlp.layers[i].get_weights()[0].shape
        weights=X[Len:(Len+mlp.layers[i].get_weights()[0].flatten().shape[0])]
        weights=np.reshape(weights,a) 
        Len=Len+mlp.layers[i].get_weights()[0].flatten().shape[0]
#         get bias first from layers
        a=mlp.layers[i].get_weights()[1].shape
        bias=X[Len:(Len+mlp.layers[i].get_weights()[1].flatten().shape[0])]
        bias=np.reshape(bias,a)
        Len=Len+mlp.layers[i].get_weights()[1].flatten().shape[0]
        mlp.layers[i].set_weights([weights,bias])
#         print(len(mlp.get_weights()))#4
#         len(mlp.layers)#4


In [8]:
def check_boundaries(X):
    for j in range(dim):
        if X[j]>U_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))
        elif  X[j]<L_bounds:
            X[j]=(L_bounds+np.random.random(1)*(U_bounds-L_bounds))

In [9]:
L_bounds=0;
U_bounds=1;
def initialization():
    for i in range(N):
        for j in range(dim):
            X[i][j]=(L_bounds+np.random.rand(1)*(U_bounds-L_bounds))
        setWeights(X[i,:])
        print("------------Solution ",i,"-------------")
        score=mlp.evaluate(X_train, y_train, verbose = 1) 
        Fit[i]= score[0]#loss
        print(score[1])#accurcy


In [10]:
#get numner node visted and probility
def pick_move(distances,pheromone, visited,alpha=1,beta=1):
    all_inds = range(len(distances))
    pheromone = np.copy(pheromone)
    # print(pheromone)
    pheromone[list(visited)] = 0
    # print(pheromone[list(visited)])
    row = pheromone ** alpha * (( 1.0 / distances) ** beta)
    # h=f"""
    # {pheromone} ** {alpha} * (( 1.0 / {dist}) ** {beta})
    # """
    # print(h)
    # print(row)
    norm_row = row / row.sum()#probility
    # print(norm_row)
    move = np_choice(all_inds, 1, p=norm_row)[0]
    return move #3

In [11]:
#get path each node
def gen_path(distances,pheromone,start=0):
    path = []
    visited = set()
    visited.add(start)
    prev = start
    for i in range(len(distances) - 1):
        move = pick_move(distances[prev],pheromone[prev], visited)
        path.append((prev, move))
        prev = move
        visited.add(move)
        # print(f"move: {move} prev:{prev} visited: {visited}")
        # print(f"-- path {path}")
    path.append((prev, start)) # going back to where we started   
    # print(f"path {path}")
    return path

In [12]:
#get distance each node
def gen_path_dist(distances,path):
    total_dist = 0
    # get values shortest path after
    total_value=list()
    for ele in path:
          total_value.append(distances[ele])
          total_dist +=distances[ele]
    #     print(f"{ele} : {distances[ele]}")
    # print(f"total_dist: {total_dist}")
    return total_dist,total_value

In [13]:
#get distance each node and path in all_path
def gen_all_paths(distances,pheromone,n_ants):
    all_paths = []
    for i in range(n_ants):
        path = gen_path(distances,pheromone)
        genPathDist,_=gen_path_dist(distances,path)
        all_paths.append((path, genPathDist))
    return all_paths

In [14]:
def spread_pheronome(distances,pheromone ,all_paths, n_best, shortest_path):
    # sorted the paths by distance
    sorted_paths = sorted(all_paths, key=lambda x: x[1])
    # print(f"sorted_paths : {sorted_paths}")
    for path, dist in sorted_paths[:n_best]:
        for move in path:
            #update pheromone values 
            pheromone[move] += 1.0 / distances[move]
    return pheromone

In [15]:
def run(distances, n_ants, n_best, n_iterations, decay):
    all_inds = range(len(distances))
    pheromone = np.ones(distances.shape) / len(distances)
    shortest_path = None
    all_time_shortest_path = ("placeholder", np.inf)
    for i in range(n_iterations):
        #get distance each node and path in all_path
        all_paths = gen_all_paths(distances,pheromone,n_ants)
        # print(f"all_paths : {all_paths}")
        pheromone=spread_pheronome(distances,pheromone ,all_paths, n_best, shortest_path=shortest_path)
        shortest_path = min(all_paths, key=lambda x: x[1])
#         print (shortest_path)
        # swap the smallest value of shortest_path
        if shortest_path[1] < all_time_shortest_path[1]:
            all_time_shortest_path = shortest_path            
        pheromone = pheromone * decay 
    return all_time_shortest_path

In [16]:
def ACO():
    initialization()
    print(X.shape)
    Max_iter=5;
    eo=list()
    bF=float('inf')
    bPos=[]
    for i in range(5):
        if Fit[i]<bF:
            bF=Fit[i]
            bPos=copy.copy(X[i]);
    t=0
    
    while t<Max_iter:
        for i in range(N):
#             NewX=copy.copy(X[i])
            distances=X[i].reshape((56,56))
#         print(distances)
            shortest_path = run(distances, 1, 1, 5, 0.95)
#             print ("shorted_path: {}".format(shortest_path))
#             print("$"*70)
#             print(shortest_path[0])
#             print("*"*40)
            _,shortest_path_value=gen_path_dist(distances,shortest_path[0])
#             print(shortest_path_value)
#             print("*"*40)
            for s in range(len(shortest_path_value)): #56
#                 print(s)
                X[i][s]=shortest_path_value[s]
#             print(len(shortest_path_value))
            
#             print("*"*40)
#             check_boundaries(NewX)
            setWeights(X[i])
            score = mlp.evaluate(X_train, y_train, verbose = 0)
            print("Accuracy:",score[1]," Loss: ",score[0])
            newFit= score[0]
            if newFit<Fit[i]:
                Fit[i]=newFit
#                 X[i]=copy.copy(X);
            if Fit[i]<bF:
                bF=Fit[i]
                bPos=copy.copy(X[i]);
        t=t+1;
        print("The best so-far Fitness value obtained by ACO at iteration ",t,"==",bF)
    eo.append(bF);
    eo.append(bPos)
    return eo;

In [17]:
runs=1
avgLoss=0
max1=float('inf')
bPos=[]
t=1
for i in range(runs):
    aco=ACO()
    avgLoss=avgLoss+aco[0]
    bPos=aco[1]

------------Solution  0 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 178990.2812 - accuracy: 0.5352
0.535240650177002
------------Solution  1 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 207131.8125 - accuracy: 0.5352
0.535240650177002
------------Solution  2 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 207310.8281 - accuracy: 0.5352
0.535240650177002
------------Solution  3 -------------
2953/2953 [==============================] - 6s 2ms/step - loss: 239594.4531 - accuracy: 0.5352
0.535240650177002
------------Solution  4 -------------
2953/2953 [==============================] - 5s 2ms/step - loss: 199463.0312 - accuracy: 0.5352
0.535240650177002
------------Solution  5 -------------
2953/2953 [==============================] - 8s 3ms/step - loss: 314875.4688 - accuracy: 0.5352
0.535240650177002
------------Solution  6 -------------
2953/2953 [==============================] - 6s 2m

Accuracy: 0.535240650177002  Loss:  75039.75
Accuracy: 0.535240650177002  Loss:  85869.4375
The best so-far Fitness value obtained by ACO at iteration  5 == 27820.17578125
